In [1]:
#!pip install yfinance --upgrade --no-cache-dir

In [1]:
#!pip freeze | grep yfinance

In [1]:
from datetime import datetime, timedelta
import sys

import pandas as pd
#from twelvedata import TDClient
#import yfinance as yf

sys.path.append('..')
import yf_clone.yfinance as yfc

sent: <object object at 0x12ff7c850>


In [2]:
#with open('./.env', 'r') as f:
#    for line in f:
#        if line.startswith('TWELVE_DATA_API_KEY'):
#            api_key = line.split('=')[-1].strip()
#            
#td_client = TDClient(apikey=api_key)

In [3]:
DATA = '../data'
TODAY = datetime.now().date()
TOMORROW  = TODAY + timedelta(1)
STATE_RANGE = range(3, 11)
START = '1965-01-01'
TOMORROW

datetime.date(2026, 1, 1)

In [4]:
def td2yf(df):
    'Convert twelvedata format to yfinance format'
    df.drop(columns=['volume'], inplace=True)
    df['Value'] = df.close.copy()
    symbols = sorted(list({i[0] for i in df.index}))
    dfs = [df.loc[sym] for sym in symbols]
    for sym, df in zip(symbols, dfs):
        df.columns = pd.MultiIndex.from_tuples(
            [(col, sym) for col in [
                'Open', 'High', 'Low', 'Close', 'Value']])
    return pd.concat(dfs, axis=1).astype(np.float64)

In [5]:
def download_td(symbols, start, end):
    df = td_client.time_series(
        symbol=symbols,
        interval='1day',
        start_date=start,
        end_date=end,
        outputsize=5000,
        timezone='Exchange',
        order='asc'
    ).as_pandas()
    df = td2yf(df)
    return df

In [6]:
#sp = download_td('^GSPC', start=START, end=str(TOMORROW))
#print(sp.shape)
#sp.head()

In [7]:
cols = ['Close', 'High', 'Low', 'Open', 'Volume']
sp = yfc.download(
    '^GSPC', start=START, end=str(TOMORROW)).sort_index()
#).sort_index()
sp.head()

YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed


Price,Close,High,Low,Open,Volume
Ticker,^GSPC,^GSPC,^GSPC,^GSPC,^GSPC
Date,,,,,
1965-01-04,84.230003,85.150002,83.769997,84.750000,3930000
1965-01-05,84.629997,85.019997,84.019997,84.230003,4110000
1965-01-06,84.889999,85.379997,84.449997,84.629997,4850000
1965-01-07,85.260002,85.620003,84.660004,84.889999,5080000
1965-01-08,85.370003,85.839996,84.910004,85.260002,5340000


In [8]:
sp.columns = cols
sp['Adj Close'] = sp.Close

path = '../data'
sp.to_csv(f'{path}/sp1950.csv')

In [9]:
#nyse = yf.download(
#    '^NYA', start=START, end=str(TOMORROW)).sort_index()
#nyse.columns = cols
#nyse['Adj Close'] = nyse.Close

nas = yfc.download(
    '^IXIC', start=START, end=str(TOMORROW)).sort_index()
nas.columns = cols
nas['Adj Close'] = nas.Close

#wil = yf.download(
#    '^W5000', start='1990-01-01', end=str(TOMORROW))

#nyse.to_csv(f'{path}/nya1965.csv')
nas.to_csv(f'{path}/nasdaq1965.csv')
#wil.to_csv(f'{path}/wilshire1990.csv')

#sp = pd.read_csv(f'{DATA}/sp1950.csv')
#nyse = pd.read_csv(f'{DATA}/nya1965.csv')
#nas = pd.read_csv(f'{DATA}/nasdaq1965.csv')

#wil = pd.read_csv(f'{DATA}/wilshire1990.csv')

[*********************100%***********************]  1 of 1 completed


In [10]:
sp.tail()

,Close,High,Low,Open,Volume,Adj Close
Date,,,,,,
2025-12-24,6932.049805,6937.319824,6904.910156,6904.910156,1798270000,6932.049805
2025-12-26,6929.939941,6945.770020,6921.600098,6936.020020,2586550000,6929.939941
2025-12-29,6905.740234,6920.209961,6888.759766,6903.600098,3541750000,6905.740234
2025-12-30,6896.240234,6913.250000,6893.470215,6900.439941,3309930000,6896.240234
2025-12-31,6845.500000,6901.419922,6844.549805,6898.819824,1709185000,6845.500000


In [11]:
nas.tail()

,Close,High,Low,Open,Volume,Adj Close
Date,,,,,,
2025-12-24,23613.310547,23621.720703,23527.970703,23555.949219,3885190000,23613.310547
2025-12-26,23593.099609,23665.150391,23567.859375,23645.910156,5170300000,23593.099609
2025-12-29,23474.349609,23531.019531,23397.519531,23414.679688,6527530000,23474.349609
2025-12-30,23419.080078,23521.050781,23414.830078,23465.669922,6769300000,23419.080078
2025-12-31,23241.990234,23445.263672,23237.783203,23420.853516,5096413000,23241.990234


In [12]:
break

SyntaxError: 'break' outside loop (668683560.py, line 1)

In [ ]:
import json

with open('/Users/damiansp/Desktop/indices.json', 'r') as f:
    stocks = json.load(f)

In [ ]:
stocks['data']

In [ ]:
[d for d in stocks['data'] if d['country'] == 'United States'
]#and 'S&P' in d['name']                
 #and d['exchange'] in ['NSE', 'NASDAQ', 'OTC']]

In [ ]:
# if yahoo finance not working us twelvedata instead
import os

from dotenv import load_dotenv
import matplotlib.pyplot as plt
import pandas as pd
from twelvedata import TDClient
from twelvedata.exceptions import BadRequestError

load_dotenv()

In [ ]:
API_KEY = os.getenv('TWELVE_DATA_API_KEY')

In [ ]:
client = TDClient(apikey=API_KEY)

In [ ]:
dfs = []
start = START
end = str(TOMORROW)

df = client.time_series(
    symbol='GSPC',  # IVV: S&P, NASDAQ
    interval='1day',
    start_date=START,
    end_date=str(TOMORROW),
    outputsize=5000,
    timezone='Exchange',
    order='asc'
).as_pandas()
dfs.append(df)

In [ ]:
end = str(dfs[-1].index[0]).split()[0]
start = datetime.fromisoformat(end) - timedelta(20 * 365)

while str(start) > START:
    print(str(start), end)
    try:
        df = client.time_series(
            symbol='IVV',  # Vanguard 500, Nasdaq Composite
            interval='1day',
            start_date=start,
            end_date=end,
            outputsize=5000,
            timezone='Exchange',
            order='asc'
        ).as_pandas()
        dfs.append(df)
    except BadRequestError:
        break
    end = str(dfs[-1].index[0]).split()[0]
    start = datetime.fromisoformat(end) - timedelta(20 * 365)

In [ ]:
df = pd.concat(dfs)
df.sort_index(inplace=True)

In [ ]:
df.head()

In [ ]:
plt.plot(df.index, df.close);

In [ ]:
end = str(df.index[0]).split()[0]
start = datetime.fromisoformat(end) - timedelta(20 * 365)
start, end

In [ ]:
data = (
    yf
    .download({'AATK', 'ABCO', 'ABAC', 'ABCW', 'AANB', 'AACC', 'ABBC'}, start='2020-01-01', end=TOMORROW)
    .rename(columns={'Adj Close': 'AdjClose'}))#['AdjClose']
data

In [ ]:
x = sp['Adj Close'].to_numpy()
np.append([np.nan], x[1:] / x[:-1])

In [ ]:
def get_daily_changes(series):
    x = series.to_numpy()
    return np.append([1], x[1:] / x[:-1])

In [ ]:
def prep_df(df, name):
    df.index = pd.to_datetime(df.index)
    df[f'{name}_daily'] = get_daily_changes(df['Adj Close'])
    df.rename(columns={'Adj Close': name}, inplace=True)
    return df[[name, f'{name}_daily']]

In [ ]:
[sp, nyse, nas, wil] = [
    prep_df(df, name) 
    for df, name in zip(
        [sp, nyse, nas, wil], ['sp', 'nyse', 'nas', 'wil'])]
sp.head()

In [ ]:
nyse.head()

In [ ]:
nas.head()

In [ ]:
def find_best_mod(
        daily,
        n_states=[6, 7, 8],
        max_iter=1000,
        reps=3,
        best_logprob=None,
        best_mod=None,
        best_states=None):
    daily = np.reshape(daily.values, [-1, 1])
    best_logprob = best_logprob or -np.inf
    best_mod = best_mod or None
    best_states = best_states or None
    for states in n_states:
        for rep in range(reps):
            try:
                print(f'rep {rep + 1}', end='\r')
                for cov in ['spherical', 'diag', 'full']:
                    mod = GaussianHMM(
                        n_components=states,
                        covariance_type=cov,
                        n_iter=max_iter
                    ).fit(daily)
                    logprob = mod.score(daily)
                    if logprob > best_logprob:
                        print(f'New best - States: {states} (cov: {cov})')
                        best_logprob = logprob
                        best_mod = mod
                        best_states = states
            except ValueError:
                pass
    return best_mod, best_logprob, best_states

In [ ]:
def get_preds(daily, mod):
    daily = np.reshape(daily.values, [-1, 1])
    states = mod.predict(daily)
    means = np.squeeze(mod.means_)
    sds = np.squeeze(np.sqrt(mod.covars_))
    preds = np.array([means[state] for state in states])
    ses = np.array([1.96 * sds[state] for state in states])
    return preds, ses, means, states[-1]

In [ ]:
def plot_mod(df, preds, ses, name):
    daily = f'{name}_daily'
    plt.plot(df[daily], label='daily')
    plt.plot(df.index, preds, label='preds')
    plt.legend()

    plt.figure()
    plt.plot(df[daily], label='daily')
    plt.plot(
        df.index, preds, label='preds', color='orange', linewidth=3)
    plt.plot(df.index, preds + ses, color='orange', linestyle='-.')
    plt.plot(df.index, preds - ses, color='orange', linestyle='-.')
    plt.axhline(y=1, color='k')
    plt.ylim([0.95, 1.05]);
    plt.xlim([df.index[-500], df.index[-1]])

    plt.figure()
    plt.plot(df[name])
    plt.xlim([df.index[-500], df.index[-1]])
    sub = df.iloc[-500:, :]
    plt.ylim(
        0.95 * sub[name].min(), 1.05 * sub[name].max());

In [ ]:
def get_expected_val(mod, current_state, means):
    return np.dot(mod.transmat_[current_state, :], means)

In [ ]:
# start search from previous day's best
with open(f'{DATA}/hmm_mods.pkl', 'rb') as f:
    out_data  = pickle.load(f)

out_data

In [ ]:
sp_prob, sp_mod, sp_best_states = (
    out_data['^GSPC']['logprob'], out_data['^GSPC']['mod'], out_data['^GSPC']['n_states'])
nyse_prob, nyse_mod, nyse_best_states = (
    out_data['^NYA']['logprob'], out_data['^NYA']['mod'], out_data['^NYA']['n_states'])
nas_prob, nas_mod, nas_best_states = (
    out_data['^IXIC']['logprob'], out_data['^IXIC']['mod'], out_data['^IXIC']['n_states'])
wil_prob, wil_mod, wil_best_states = (
    out_data['^W5000']['logprob'], out_data['^W5000']['mod'], out_data['^W5000']['n_states'])

In [ ]:
type(sp_mod)

## S&P

In [ ]:
warnings.filterwarnings('ignore', category=RuntimeWarning)

In [ ]:
#sp_prob = None
#sp_mod = None
#sp_best_states = None

In [ ]:
sp_mod, sp_prob, sp_best_states = find_best_mod(
    sp.sp_daily,
    n_states=STATE_RANGE,
    best_logprob=sp_prob,
    best_mod=sp_mod,
    best_states=sp_best_states)
sp_preds, sp_ses, sp_means, sp_current = get_preds(sp.sp_daily, sp_mod)
plot_mod(sp, sp_preds, sp_ses, 'sp')
sp_exp = get_expected_val(sp_mod, sp_current, sp_means)
print(sp_exp)

## NYSE

In [ ]:
#nyse_prob = None
#nyse_mod = None
#nyse_best_states = None

In [ ]:
nyse_mod, nyse_prob, nyse_best_states = find_best_mod(
    nyse.nyse_daily,
    n_states=STATE_RANGE,
    best_logprob=nyse_prob,
    best_mod=nyse_mod,
    best_states=nyse_best_states)
nyse_preds, nyse_ses, nyse_means, nyse_current = get_preds(
    nyse.nyse_daily, nyse_mod)
plot_mod(nyse, nyse_preds, nyse_ses, 'nyse')
nyse_exp = get_expected_val(nyse_mod, nyse_current, nyse_means)
print(nyse_exp)

## Nasdaq

In [ ]:
#nas_prob = None
#nas_mod = None
#nas_best_states = None

In [ ]:
nas_mod, nas_prob, nas_best_states = find_best_mod(
    nas.nas_daily,
    n_states=STATE_RANGE,
    best_logprob=nas_prob,
    best_mod=nas_mod,
    best_states=nas_best_states)
nas_preds, nas_ses, nas_means, nas_current = get_preds(
    nas.nas_daily, nas_mod)
plot_mod(nas, nas_preds, nas_ses, 'nas')
nas_exp = get_expected_val(nas_mod, nas_current, nas_means)
print(nas_exp)

## Wilshire

In [ ]:
#wil_prob = None
#wil_mod = None
#wil_best_states = None

In [ ]:
wil_mod, wil_prob, wil_best_states = find_best_mod(
    wil.wil_daily,
    n_states=STATE_RANGE,
    best_logprob=wil_prob,
    best_mod=wil_mod,
    best_states=wil_best_states)
wil_preds, wil_ses, wil_means, wil_current = get_preds(
    wil.wil_daily, wil_mod)
plot_mod(wil, wil_preds, wil_ses, 'wil')
wil_exp = get_expected_val(wil_mod, wil_current, wil_means)
print(wil_exp)

In [ ]:
#sp_best_states = len(np.unique(sp_preds))
#nyse_best_states = len(np.unique(nyse_preds))
#nas_best_states = len(np.unique(nas_preds))
#wil_best_states = len(np.unique(wil_preds))

In [ ]:
out_data = {
    '^GSPC': {'logprob': sp_prob, 'mod': sp_mod, 'n_states': sp_best_states},
    '^NYA': {'logprob': nyse_prob, 'mod': nyse_mod, 'n_states': nyse_best_states},
    '^IXIC': {'logprob': nas_prob, 'mod': nas_mod, 'n_states': nas_best_states},
    '^W5000': {'logprob': wil_prob, 'mod': wil_mod, 'n_states': wil_best_states}}
with open(f'{DATA}/hmm_mods_indices.pkl', 'wb') as f:
    pickle.dump(out_data, f)

In [ ]:
print(
    sp_best_states, nyse_best_states, nas_best_states, wil_best_states)
mean_states = (
    (sp_best_states
     + nyse_best_states
     + nas_best_states
     + wil_best_states)
    / 4)
mean_states

In [ ]:
N_STATES = int(round(mean_states))
N_STATES

In [ ]:
exp = (sp_exp + nyse_exp + nas_exp + wil_exp) / 4
exp

In [ ]:
for df, name, exp in zip(
        [sp, nyse, nas, wil],
        ['sp', 'nyse', 'nas', 'wil'],
        [sp_preds, nyse_preds, nas_preds, wil_preds]):
    df[f'{name}_exp'] = exp

In [ ]:
df = pd.concat([sp, nyse, nas, wil], axis=1)
df.tail()

In [ ]:
df['mean_exp'] = (
    df[['sp_exp', 'nyse_exp', 'nas_exp', 'wil_exp']].mean(axis=1))

In [ ]:
plt.figure(figsize=[15, 8])
plt.plot(df.sp_exp, label='sp')
plt.plot(df.nyse_exp, label='nyse')
plt.plot(df.nas_exp, label='nasdaq')
plt.plot(df.wil_exp, label='wilshire')

plt.legend();

In [ ]:
plt.figure(figsize=[15, 8])
plt.plot(df.sp_exp, alpha=0.5)
plt.plot(df.nyse_exp, alpha=0.5)
plt.plot(df.nas_exp, alpha=0.5)
plt.plot(df.wil_exp, alpha=0.5)
plt.plot(df.mean_exp, linewidth=3)
plt.axhline(y=1, color='k');

In [ ]:
TODAY = datetime.now()
plt.figure(figsize=[15, 8])
plt.plot(df.sp_exp, alpha=0.5)
plt.plot(df.nyse_exp, alpha=0.5)
plt.plot(df.nas_exp, alpha=0.5)
plt.plot(df.wil_exp, alpha=0.5)
plt.plot(df.mean_exp, linewidth=3)
plt.axhline(y=1, color='k')
plt.xlim(pd.to_datetime('2010-01-01'), TODAY);

In [ ]:
TODAY = datetime.now()
plt.figure(figsize=[15, 8])
plt.plot(df.sp_exp, alpha=0.5)
plt.plot(df.nyse_exp, alpha=0.5)
plt.plot(df.nas_exp, alpha=0.5)
plt.plot(df.wil_exp, alpha=0.5)
plt.plot(df.mean_exp, linewidth=3)
plt.xlim(pd.to_datetime('2020-01-01'), TODAY);
plt.axhline(y = 1, color='k');

In [ ]:
plt.figure(figsize=[15, 8])
plt.plot(df.sp_exp, alpha=0.5)
plt.plot(df.nyse_exp, alpha=0.5)
plt.plot(df.nas_exp, alpha=0.5)
plt.plot(df.wil_exp, alpha=0.5)
plt.plot(df.mean_exp, linewidth=3)
plt.axhline(y=1, color='k')
plt.xlim(pd.to_datetime('2021-01-01'), TODAY)
plt.ylim([0.995, 1.005]);

In [ ]:
df = df[df.index >= pd.to_datetime('1970-01-01')]

In [ ]:
try:
    df['state'] = pd.qcut(
        df.mean_exp,
        N_STATES,
        retbins=False,
        duplicates='drop',
        labels=range(N_STATES))
except ValueError:
    try:
        df['state'] = pd.qcut(
            df.mean_exp,
            N_STATES,
            retbins=False,
            duplicates='drop',
            labels=range(N_STATES - 1))
    except ValueError:
        df['state'] = pd.qcut(
            df.mean_exp,
            N_STATES,
            retbins=False,
            duplicates='drop',
            labels=range(N_STATES - 2))

In [ ]:
df.state.value_counts()

In [ ]:
plt.figure(figsize=[15, 8])
plt.plot(df.state);

In [ ]:
plt.figure(figsize=[15, 8])
plt.plot(df.state)
plt.xlim(pd.to_datetime('2010-01-01'), TODAY);

plt.figure(figsize=[15, 8])
plt.plot(df.state)
plt.xlim(pd.to_datetime('2021-01-01'), TODAY);

plt.figure(figsize=[15, 8])
plt.plot(df.state)
plt.xlim(pd.to_datetime('2022-01-01'), TODAY);

In [ ]:
transitions = Counter()
for i in range(len(df.state) - 1):
    state = df.state[i]
    nxt = df.state[i + 1]
    transitions[(state, nxt)] += 1

In [ ]:
df.to_csv(f'{DATA}/hmm_exp_returns.csv')

In [ ]:
TODAY

In [ ]:
CURRENT_STATE = df.state[-1]
CURRENT_STATE

In [ ]:
possible_transitions = sorted(
    [k for k in transitions.keys() if k[0] == CURRENT_STATE])
state_sum = 0
for pt in possible_transitions:
    n_trans = transitions[pt]
    state_sum += n_trans
    print(f'{pt}: {n_trans}')

print()

data = []
for pt in possible_transitions:
    n_trans = transitions[pt]
    print(f'{pt}: {n_trans / state_sum:.4f}')
    data.append([pt[1], n_trans / state_sum])

In [ ]:
trans_df = pd.DataFrame(data, columns=['next_state', 'prob'])
trans_df

In [ ]:
trans_df.to_csv('../data/transition_probs.csv', index=False)

In [ ]:
df.state.value_counts()

In [ ]:
state_var = (
    df[['state', 'mean_exp']].groupby('state').agg(['mean', 'std']))
state_var.columns = ['mean', 'std']
state_var
state_var['ci'] = state_var.apply(
    lambda row: [
        row['mean'] - 1.96*row['std'],
        row['mean'] + 1.96*row['std']], axis=1)
state_var

In [ ]:
plt.scatter(state_var.index, state_var['mean'])
for state in range(state_var.index.max() + 1):
    plt.plot([state, state], state_var.loc[state, 'ci'], color='k');

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
s = pd.Series([np.nan, np.nan, 1, 3, 4, np.nan, np.nan, 7, 8, np.nan, 10, np.nan, np.nan])
s

In [ ]:
not_null_idxs = s.notnull()[s.notnull()].index
valid_start = not_null_idxs[0]
valid_end = not_null_idxs[-1]
s[valid_start:valid_end] = s[valid_start:valid_end].fillna(method='ffill')
s

In [ ]:
symbols = ['AA', 'AABA', 'TSLA']
data = yf.download(symbols, start='1962-01-01', end='2024-03-23').drop(columns='Open')
data.head()

In [ ]:
new_cols = data.columns.map(lambda t: f'{t[1]}_{t[0]}'.lower().replace('_adj close', ''))
new_cols

In [ ]:
data.columns = new_cols
data.head()

In [ ]:
data.index = data.index.astype(str)
data.head()

In [ ]:
n = len(data)
nullsum = data.isnull().sum()
empties = nullsum[nullsum == n].index
empties

In [ ]:
data.drop(columns=empties, inplace=True)
data.head()

In [ ]:
def adj(sym, close, hilo):
    factor = sym / close
    #return factor
    hilo *= factor
    return hilo

In [ ]:
adj(data.aa, data.aa_close, data.aa_low)

In [ ]:
adj(data.aa, data.aa_close, data.aa_high)

In [ ]:
def get_pct_change(stock: pd.Series) -> pd.Series:
    return pd.Series(
        np.concatenate([[1], stock.values[1:] / stock.values[:-1]]), index=stock.index)

In [ ]:
get_pct_change(data.a)

In [ ]:
def get_next_day_ranges(close: pd.Series, low: pd.Series, high: pd.Series):
        c = close.values[:-1]
        l = low.values[1:]
        h = high.values[1:]
        next_day_min = pd.Series(np.concatenate([[1], l / c]), index=close.index)
        next_day_max = pd.Series(np.concatenate([[1], h / c]), index=close.index)
        return next_day_min, next_day_max                                     

In [ ]:
# Test
close = pd.Series([  1,   2,   3,   4, 3.7])
low =   pd.Series([0.8, 1.8, 1.9, 4.1, 4.7])
high =  pd.Series([0.9, 2.2, 3.3, 4.4, 3.9])
get_next_day_ranges(close, low, high)

In [ ]:
import numpy as np


def random_password(n):
    opt = 'abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQSRTUVWXYZ`1234567890-=[]\\;,./~!@#$%^&*()_+{}|:"<>?\'"'
    return ''.join(np.random.choice(list(opt), size=n, replace=True))

In [ ]:
random_password(20)

In [15]:
def convert_value(s):
    return round(float(s.replace('$', '').replace(',', '')))

In [20]:
def separate_accounts(fid):
    accounts = {
        account_name: fid[fid['Account Name'] == account_name]
        for account_name in fid['Account Name'].unique()}
    for acct in accounts:
        accounts[acct].index = accounts[acct].Symbol
        accounts[acct] = (
            accounts[acct]
            .rename(columns={'Current Value': acct})
            .drop(columns=['Account Name', 'Symbol']))
    return accounts

In [23]:
fidelity = pd.read_csv('~/Downloads/Portfolio_Positions_Dec-09-2025.csv')
#fidelity = fidelity[fidelity['Last Price'].notnull()]
#fidelity = fidelity[fidelity['Last Price'] != '--']
#fidelity = (fidelity[['Account Name', 'Symbol', 'Current Value']].dropna())
fidelity.head()

,Account Number,Account Name,Symbol,Description,Quantity,Last Price,Last Price Change,Current Value,Today's Gain/Loss Dollar,Today's Gain/Loss Percent,Total Gain/Loss Dollar,Total Gain/Loss Percent,Percent Of Account,Cost Basis Total,Average Cost Basis,Type
218520300,Rollover IRA,SPAXX**,HELD IN MONEY MARKET,NaN,NaN,NaN,$12204.09,NaN,NaN,NaN,NaN,15.42%,NaN,NaN,Cash,NaN
218520300,Rollover IRA,003CVR016,ABIOMED INC SHS,4.0,--,--,--,--,--,--,--,0.00%,--,--,Cash,NaN
218520300,Rollover IRA,RACE,FERRARI N V EUR0.01 NEW,32.0,$375.35,-$7.63,$12011.20,-$244.16,-2.00%,-$960.97,-7.41%,15.18%,$12972.17,$405.38,Cash,NaN
218520300,Rollover IRA,PSTG,PURE STORAGE INC CL A,88.0,$70.15,-$0.89,$6173.20,-$78.32,-1.26%,+$150.48,+2.49%,7.80%,$6022.72,$68.44,Cash,NaN
218520300,Rollover IRA,VEEV,VEEVA SYSTEMS INC,11.0,$232.00,-$7.05,$2552.00,-$77.55,-2.95%,-$41.14,-1.59%,3.23%,$2593.14,$235.74,Cash,NaN


In [17]:
fidelity['Current Value'] = (
    fidelity['Current Value']
    .apply(convert_value)
    .fillna(0)
    .astype(int))
fidelity.head()

,Account Name,Symbol,Current Value
218520300,RACE,FERRARI N V EUR0.01 NEW,-244
218520300,PSTG,PURE STORAGE INC CL A,-78
218520300,VEEV,VEEVA SYSTEMS INC,-78
218520300,HD,HOME DEPOT INC,-70
218520300,DOCS,DOXIMITY INC CL A,-49


In [21]:
fidelity = fidelity[
    ((fidelity.Symbol != 'SPAXX**')
     & (fidelity.Symbol != 'Pending Activity'))]
fidelity = separate_accounts(fidelity)
fidelity

{'RACE':                          RACE
 Symbol                       
 FERRARI N V EUR0.01 NEW  -244
 FERRARI N V EUR0.01 NEW   -92,
 'PSTG':                        PSTG
 Symbol                     
 PURE STORAGE INC CL A   -78,
 'VEEV':                    VEEV
 Symbol                 
 VEEVA SYSTEMS INC   -78
 VEEVA SYSTEMS INC  -381,
 'HD':                 HD
 Symbol            
 HOME DEPOT INC -70
 HOME DEPOT INC -46,
 'DOCS':                    DOCS
 Symbol                 
 DOXIMITY INC CL A   -49,
 'LEU':                      LEU
 Symbol                  
 CENTRUS ENERGY CORP  -46,
 'PTC':          PTC
 Symbol      
 PTC INC  -27
 PTC INC   -4,
 'TYL':                         TYL
 Symbol                     
 TYLER TECHNOLOGIES INC  -14
 TYLER TECHNOLOGIES INC   -7,
 'SERV':                         SERV
 Symbol                      
 SERVE ROBOTICS INC COM   -12,
 'META':                                          META
 Symbol                                       
 META PLATFORMS 